# Otimização do Serviço de Ônibus para Embarque Remoto

Este notebook apresenta a **formulação matemática** e a **implementação da resolução** do problema de otimização considerado.

## Descrição do Problema

O problema consiste em definir as rotas dos ônibus utilizados para levar passageiros do portão de embarque até o avião ou do avião até o portão de embarque. Essa rota deve minimizar os custos de transporte que, nesse caso, será interpretado como equivalente à distância total percorrida. Os ônibus têm capacidades iguais e cada grupo de passageiros de cada voo será desmembrado em requisições. Logo, o modelo será escrito para determinar as sequência de requisições que cada ônibus deve fazer.

### Características Principais:
- **Janelas de tempo** específicas para cada requisição
- **Frota limitada** de ônibus
- **Restrições de autonomia** (distância máxima por viagem)
- **Múltiplas viagens** por ônibus durante o período de operação

## 1. Importação de Bibliotecas e Carregamento dos Dados

Iniciamos importando as bibliotecas necessárias e carregando uma instância do problema:

In [14]:
import gurobipy as gp
from gurobipy import GRB
from time import time
from dados import carrega_dados_json
from solucao import Solucao

nome_arquivo = "./dados/pequena.json"
dados = carrega_dados_json(nome_arquivo)

modelo = gp.Model("Otimização do Serviço de Ônibus para Embarque Remoto")

## 2. Definição dos Conjuntos e Constantes

O problema é modelado como um **Problema de Programação Linear Inteira Mista** similar ao problema de roteamento de veículos com coleta e entrega acoplados, janelas de tempo, múltiplas viagens e distância máxima. O problema possui as seguintes características:

### Conjuntos:
- **N**: Conjunto de requisições {1, 2, ..., n}
- **N₀**: Conjunto de requisições incluindo a garagem {0, 1, 2, ..., n}
- **K**: Conjunto de ônibus {1, 2, ..., k}
- **V**: Conjunto de viagens possíveis por ônibus {1, 2, ..., r}

### Parâmetros do Modelo:

- **D[i,j]**: Distância entre as requisições i e j
- **c[i,j]**: Custo de viagem entre as requisições i e j  
- **T[i,j]**: Tempo de viagem entre as requisições i e j
- **s[i]**: Tempo de serviço na requisição i
- **e[i], l[i]**: Início e fim da janela de tempo da requisição i
- **Dmax**: Distância máxima que um ônibus pode percorrer por viagem

In [15]:
N = list(range(1, dados.n+1))
N0 = list(range(dados.n+1))
V = list(range(1, dados.r+1))
K = list(range(1, dados.K+1))
x = {}
y = {}
B = {}
M = 1e10
LIMITE_TEMPO = 5*60.

## 3. Variáveis de Decisão

O modelo utiliza três tipos de variáveis de decisão:

### **Variáveis Binárias:**

**x[i,j,v,k]**: Variável binária que indica se o ônibus k na viagem v percorre o arco de i para j
- **x[i,j,v,k] = 1**: O ônibus k vai da requisição i para j na viagem v
- **x[i,j,v,k] = 0**: Caso contrário

**y[v,k]**: Variável binária que indica se o ônibus k realiza a viagem v
- **y[v,k] = 1**: O ônibus k executa a viagem v
- **y[v,k] = 0**: Caso contrário

### **Variáveis Contínuas:**

**B[i,v,k]**: Tempo de chegada do ônibus k na requisição i durante a viagem v

In [16]:
for k in K:
    for v in V:
        for i in N0:
            for j in N0:
                if i != j:
                    x[i, j, v, k] = modelo.addVar(vtype=GRB.BINARY,
                                            name=f"x_{i}_{j}_{v}_{k}")

            B[i, v, k] = modelo.addVar(vtype=GRB.CONTINUOUS,
                                        lb=0.0,
                                        name=f"B_{i}_{v}_{k}")
            
        y[v, k] = modelo.addVar(vtype=GRB.BINARY,
                                name=f"y_{v}_{k}")
modelo.update()

## 4. Função Objetivo

**Minimizar o custo total de operação dos ônibus:**

$$\min \sum_{i \in N_0} \sum_{j \in N_0} \sum_{v \in V} \sum_{k \in K} c_{ij} \cdot x_{ijvk}$$

onde:
- **c[i,j]**: Custo de percorrer o arco da requisição i para j
- **x[i,j,v,k]**: Variável binária de roteamento

O objetivo é **minimizar a soma de todos os custos** dos arcos utilizados por todos os ônibus em todas as viagens.

In [17]:
funcao_objetivo = modelo.setObjective(
    gp.quicksum(dados.c[i, j] * x[i, j, v, k] 
                for i in N0 
                for j in N0
                for v in V
                for k in K
                if i != j),
    GRB.MINIMIZE
)

## 5. Restrições do Modelo

### 5.1. Restrição de Atendimento

**Cada requisição deve ser atendida exatamente uma vez:**

$$\sum_{i \in N_0} \sum_{k \in K} \sum_{v \in V} x_{ijvk} = 1, \quad \forall j \in N$$

Esta restrição garante que:
- **Toda requisição é atendida** 
- **Nenhuma requisição é atendida mais de uma vez**
- **Não há requisições não atendidas**

In [18]:
for j in N:
    modelo.addConstr(
        gp.quicksum(x[i, j, v, k]
                    for i in N0
                    for k in K
                    for v in V
                    if i != j) == 1,
        name=f"atendimento_{j}"
)

### 5.2. Restrições de Conservação de Fluxo

**O que entra deve sair - continuidade das rotas:**

$$\sum_{i \in N_0, i \neq j} x_{ijvk} - \sum_{i \in N_0, i \neq j} x_{jivk} = 0, \quad \forall j \in N, \forall v \in V, \forall k \in K$$

Esta restrição garante que:
- **Se um ônibus chega** a uma requisição j, **ele deve sair** dela
- **Continuidade das rotas** sem interrupções
- **Estrutura de caminho** válida no grafo

In [19]:
for k in K:
    for v in V:
        for j in N:

            entrada = gp.quicksum(x[i, j, v, k]
                                for i in N0
                                if i != j)

            saida = gp.quicksum(x[j, i, v, k]
                                for i in N0
                                if i != j)

            modelo.addConstr(
                entrada - saida == 0,
                name=f"conservacao_{j}_{v}_{k}"
            )


### 5.3. Restrições de Início e Término de Viagens

**Toda viagem começa e termina na garagem:**

**Início da viagem:**
$$\sum_{j \in N} x_{0jvk} = y_{vk}, \quad \forall v \in V, \forall k \in K$$

**Término da viagem:**
$$\sum_{i \in N} x_{i0vk} = y_{vk}, \quad \forall v \in V, \forall k \in K$$

Estas restrições garantem que:
- **Toda viagem ativa** sai da garagem (nó 0)
- **Toda viagem ativa** retorna à garagem
- **Viagens inativas** não têm movimento
- **Consistência** entre variáveis x e y

In [20]:
for k in K:
    for v in V:
        modelo.addConstr(
            gp.quicksum(x[0, j, v, k]
                        for j in N) == y[v, k],
            name=f"inicio_viagem_{v}_{k}"
        )
        modelo.addConstr(
            gp.quicksum(x[i, 0, v, k]
                        for i in N) == y[v, k],
            name=f"termino_viagem_{v}_{k}"
        )

### 5.4. Restrições de Sequenciamento de Viagens

**As viagens devem ser executadas em ordem sequencial:**

$$y_{vk} \leq y_{(v-1)k}, \quad \forall v \in V, v > 1, \forall k \in K$$

Esta restrição garante que:
- **Viagem v só pode ser ativa** se a viagem (v-1) também for ativa
- **Ordem cronológica** das viagens
- **Não há gaps** entre viagens (ex: viagem 1 e 3 ativas, mas 2 inativa)
- **Estrutura sequencial** lógica

In [21]:
for k in K:
    for v in V:
        if v > 1:
            modelo.addConstr(
                y[v, k] <= y[v-1, k],
                name=f"sequencia_viagem_{v}_{k}"
            )

### 5.5. Restrições de Distância Máxima

**Cada viagem respeita a autonomia do ônibus:**

$$\sum_{i \in N_0} \sum_{j \in N_0} D_{ij} \cdot x_{ijvk} \leq D_{max}, \quad \forall v \in V, \forall k \in K$$

Esta restrição garante que:
- **Autonomia dos ônibus** é respeitada
- **Nenhuma viagem excede** a distância máxima permitida
- **Operação segura** sem risco de pane por combustível
- **Viabilidade prática** das rotas

In [22]:
for k in K:
    for v in V:
        modelo.addConstr(
            gp.quicksum(dados.D[i,j] * x[i, j, v, k]
                        for i in N0
                        for j in N0
                        if i != j) <= dados.Dmax,
            name=f"distancia_maxima_{v}_{k}"
        )

### 5.6. Restrições de Janelas de Tempo

**O atendimento deve ocorrer dentro do período especificado:**

**Limite inferior:**
$$B_{ivk} \geq e_i \cdot \sum_{j \in N_0, j \neq i} x_{jivk}, \quad \forall i \in N, \forall v \in V, \forall k \in K$$

**Limite superior:**
$$B_{ivk} \leq l_i \cdot \sum_{j \in N_0, j \neq i} x_{jivk}, \quad \forall i \in N, \forall v \in V, \forall k \in K$$

Estas restrições garantem que:
- **Atendimento não antecipado**: B[i,v,k] ≥ e[i]
- **Atendimento não atrasado**: B[i,v,k] ≤ l[i]  
- **Respeito aos horários** dos voos
- **Coordenação temporal** das operações

In [23]:
for k in K:
    for v in V:
        for i in N:
            modelo.addConstr(
                B[i, v, k] >= dados.e[i-1] * gp.quicksum(x[j, i, v, k]
                for j in N0 if i != j),
                name=f"janela_inferior_{i}_{v}_{k}"
            )
            modelo.addConstr(
                B[i, v, k] <= dados.l[i-1] * gp.quicksum(x[j, i, v, k]
                for j in N0 if i != j),
                name=f"janela_superior_{i}_{v}_{k}"
            )

### 5.7. Restrições de Fluxo de Tempo

**Coordenação temporal entre requisições e viagens:**

#### **Fluxo Intra-viagem** (dentro da mesma viagem):
$$B_{ivk} + s_i + T_{ij} - M \cdot (1 - x_{ijvk}) \leq B_{jvk}$$

#### **Fluxo Inter-viagem** (entre viagens consecutivas):
$$B_{0(v-1)k} + s_0 + T_{0i} - M \cdot (1 - x_{0ivk}) \leq B_{ivk}$$

Estas restrições garantem que:
- **Sequência temporal** correta dentro de cada viagem
- **Tempo de serviço** é considerado em cada requisição
- **Tempo de deslocamento** entre requisições é respeitado
- **Continuidade temporal** entre viagens do mesmo ônibus
- **Tempo de reabastecimento** na garagem entre viagens

Onde **M** é uma constante suficientemente grande (Big-M) para tornar a restrição inativa quando x=0.

In [24]:
for k in K:
    for v in V:
        for i in N0:
            for j in N0:
                
                if i == j:
                    continue
                
                elif i == 0 and v == 1:
                    modelo.addConstr(
                        dados.s[0] + dados.T[0, j] - M * (1 - x[0, j, 1, k]) 
                        <= B[j, 1, k],
                        name=f"fluxo_tempo_intra_0_{j}_{v}_{k}"
                    )

                else:
                    modelo.addConstr(
                        B[i, v, k] + dados.s[i] + dados.T[i, j]
                        - M * (1 - x[i, j, v, k]) <= B[j, v, k],
                    name=f"fluxo_tempo_intra_{i}_{j}_{v}_{k}"
                )
            
            if v > 1 and i != 0:
                modelo.addConstr(
                    B[0, v-1, k] + dados.s[0] + dados.T[0, i] 
                    - M * (1 - x[0, i, v, k]) <= B[i, v, k],
                    name=f"fluxo_tempo_inter_{i}_{v}_{k}"
                )

## 6. Resolução do Modelo com Gurobi

Após definir todas as variáveis, função objetivo e restrições, executamos a otimização:

In [25]:
modelo.update()
modelo.setParam(GRB.Param.TimeLimit, LIMITE_TEMPO)
tic = time()
modelo.optimize()
tempo_total = time() - tic
if modelo.Status == GRB.OPTIMAL:
    print(f"Solução ótima encontrada em {tempo_total:.2f} segundos.")
elif modelo.Status == GRB.TIME_LIMIT:
    print(f"Tempo limite atingido em {tempo_total:.2f} segundos.")
else:
    print(f"Solução não encontrada. Status: {modelo.Status}")
    exit()

Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-4710HQ CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  300

Optimize a model with 5479 rows, 4812 columns and 41082 nonzeros
Model fingerprint: 0xc1efb28c
Variable types: 240 continuous, 4572 integer (4572 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+10]
  Objective range  [4e+02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+10]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 4153 rows and 2789 columns
Presolve time: 0.54s
Presolved: 1326 rows, 2023 columns, 22079 nonzeros
Variable types: 189 continuous, 1834 integer (1825 binary)

Root relaxation: objective 4.501444e+04, 1333 iterations, 0.05 seconds (0.05 wo

## 7. Apresentação da Solução

A classe `Solucao` processa os resultados do Gurobi e apresenta:

In [26]:
solucao = Solucao()
solucao.carrega_modelo_gurobi(modelo, dados)
print(solucao)

Ônibus 1: [0, 1, 3, 7, 10, 11, 13, 15, 17, 18, 0]
Ônibus 2: [0, 4, 6, 8, 12, 0]
Ônibus 3: [0, 2, 5, 9, 14, 16, 19, 0]
